<br><br>
<center><font size='5'><b>Deep Learning for All_pytorch</b></font><br><br><font size='5'>Chap2. Linear Regression<b></b></font>

# Simple Linear Regression

## Training data, Weight, biases 정의

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

In [4]:
# Weight과 Bias를 0으로 초기화 -> 항상 출력 0으로 예측
# requires_grad = True : 학습할 것이라고 명시

W = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

hypothesis = x_train*W + b

## Variable을 사용한 정의

In [56]:
from torch.autograd import Variable

In [58]:
W = Variable(torch.zeros(1), requires_grad=True)

## Compute loss 

### Mean Squared Error(MSE)

In [5]:
cost = torch.mean((hypothesis - y_train)**2)

### Gradient Descent

- torch.optim 라이브러리 사용
  - [W,b]는 학습할 tensor들
  - lr은 learning rate

In [ ]:
optimizer = torch.optim.SGD([W,b], lr=0.01)


#######################항상 붙어다니는 3줄##############################
optimizer.zero_grad() # Gradient 초기화
cost.backward() # backward로 gradient 계산
optimizer.step() # step()으로 개선

## 총정리 

- 한번만
  - 데이터 정의, hypothesis 초기화, optimizer 정의
- iteration
  - Hypothesis 예측, cost 계산, optimizer로 학습

In [11]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


In [11]:
print(W)
print(b)

tensor([1.7078], requires_grad=True)
tensor([0.6515], requires_grad=True)


## GD 직접 구현해 보기 

In [8]:
# 데이터
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[1],[2],[3]])

# 모델 초기화
W = torch.zeros(1)

# learning rate 설정
lr = 0.1

n_epochs = 10
for epoch in range(1, n_epochs+1):
    
    # H(x) 계산
    hypothesis = x_train*W
    
    # cost gradient 계산
    cost = torch.mean((hypothesis - y_train)**2)
    gradient = torch.sum((W*x_train - y_train) * x_train)
    
    print('Epoch {:4d}/{} W: {:.3f}, Cost: {:.6f}'.format(epoch, n_epochs, W.item(), cost.item()))
    
    # cost gradient로 H(x) update
    W -= lr * gradient

Epoch    1/10 W: 0.000, Cost: 4.666667
Epoch    2/10 W: 1.400, Cost: 0.746666
Epoch    3/10 W: 0.840, Cost: 0.119467
Epoch    4/10 W: 1.064, Cost: 0.019115
Epoch    5/10 W: 0.974, Cost: 0.003058
Epoch    6/10 W: 1.010, Cost: 0.000489
Epoch    7/10 W: 0.996, Cost: 0.000078
Epoch    8/10 W: 1.002, Cost: 0.000013
Epoch    9/10 W: 0.999, Cost: 0.000002
Epoch   10/10 W: 1.000, Cost: 0.000000


## High-level Implementation with nn.Module

In [12]:
# data
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

- 모델의 __init__에서는 사용할 레이어들을 정의하게 된다. 
- forward에서는 이 모델이 어떻게 입력값에서 출력값을 계산하는지 알려준다.

In [33]:
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [34]:
model = LinearRegressionModel()

In [35]:
hypothesis = model(x_train)

In [36]:
print(hypothesis)

tensor([[ 0.3905],
        [ 0.1615],
        [-0.0676]], grad_fn=<AddmmBackward>)


In [37]:
# MSE로 cost 구하기

cost = F.mse_loss(hypothesis, y_train)

In [38]:
# GradientDescent

optimizer = optim.SGD(model.parameters(), lr=0.01)

In [39]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

### full code

In [40]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 1.008, b: -0.590 Cost: 0.406030
Epoch  100/1000 W: 1.172, b: -0.391 Cost: 0.022032
Epoch  200/1000 W: 1.135, b: -0.307 Cost: 0.013615
Epoch  300/1000 W: 1.106, b: -0.242 Cost: 0.008413
Epoch  400/1000 W: 1.084, b: -0.190 Cost: 0.005199
Epoch  500/1000 W: 1.066, b: -0.149 Cost: 0.003212
Epoch  600/1000 W: 1.052, b: -0.117 Cost: 0.001985
Epoch  700/1000 W: 1.041, b: -0.092 Cost: 0.001227
Epoch  800/1000 W: 1.032, b: -0.073 Cost: 0.000758
Epoch  900/1000 W: 1.025, b: -0.057 Cost: 0.000468
Epoch 1000/1000 W: 1.020, b: -0.045 Cost: 0.000289


# Multivariate Linear Regression

## 기존 방법

![image](https://user-images.githubusercontent.com/48466625/61097625-f68d0f00-a496-11e9-8495-0b9244c96a2a.png)

$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$

In [1]:
import torch

x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,80],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

In [2]:
# Weight, biases 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [4]:
# Optimizer 설정

optimizer = torch.optim.SGD([W,b], lr=1e-5)

In [9]:
n_epochs = 20

for epoch in range(n_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b
    
    # Cost 계산
    cost = torch.mean((hypothesis - y_train)**2)
    
    # cost로 H(x) 업데이트
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} W: {}, Cost: {:.6f}'.format(epoch, n_epochs, hypothesis.squeeze().detach(), cost.item()))


Epoch    0/20 W: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]), Cost: 322.482056
Epoch    1/20 W: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]), Cost: 107.717064
Epoch    2/20 W: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]), Cost: 38.687500
Epoch    3/20 W: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]), Cost: 16.499043
Epoch    4/20 W: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]), Cost: 9.365657
Epoch    5/20 W: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]), Cost: 7.071114
Epoch    6/20 W: tensor([153.5285, 184.4835, 175.1338, 197.9415, 140.7325]), Cost: 6.331847
Epoch    7/20 W: tensor([153.7572, 184.7582, 175.3958, 198.2360, 140.9424]), Cost: 6.092532
Epoch    8/20 W: tensor([153.8868, 184.9138, 175.5449, 198.4026, 141.0613]), Cost: 6.013817
Epoch    9/20 W: tensor([153.9602, 185.0019, 175.6299, 198.4969, 141.1288]), Cost: 5.986785
Epoch   10/20 W: tensor([154.0017, 185.0517, 175.6785, 198.5500, 141.1671]

## nn.Module

In [41]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력차원은 3이고 출력차원은 1

    def forward(self, x):
        return self.linear(x) # hypothesis 계산을 어떻게 할지 알려주는

In [42]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 14229.998047
Epoch    1/20 Cost: 4460.848633
Epoch    2/20 Cost: 1398.737671
Epoch    3/20 Cost: 438.928680
Epoch    4/20 Cost: 138.079956
Epoch    5/20 Cost: 43.779396
Epoch    6/20 Cost: 14.221448
Epoch    7/20 Cost: 4.956458
Epoch    8/20 Cost: 2.052372
Epoch    9/20 Cost: 1.142069
Epoch   10/20 Cost: 0.856727
Epoch   11/20 Cost: 0.767262
Epoch   12/20 Cost: 0.739204
Epoch   13/20 Cost: 0.730392
Epoch   14/20 Cost: 0.727615
Epoch   15/20 Cost: 0.726716
Epoch   16/20 Cost: 0.726412
Epoch   17/20 Cost: 0.726307
Epoch   18/20 Cost: 0.726243
Epoch   19/20 Cost: 0.726196
Epoch   20/20 Cost: 0.726168


# 데이터 생성하기

In [59]:
# 신경망에 사용할 데이터 생성

def get_data(): ###### 입력과 출력 데이터로 구성된 Tensor 배열 제공
    train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
    train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
    dtype = torch.FloatTensor
    X = Variable(torch.from_numpy(train_X).type(dtype),requires_grad=False).view(17,1)
    y = Variable(torch.from_numpy(train_Y).type(dtype),requires_grad=False)
    return X,y

In [61]:
# 학습 파라미터 생성

def get_weights(): #### 최적화할 파라미터 
    w = Variable(torch.randn(1),requires_grad = True)
    b = Variable(torch.randn(1),requires_grad=True)
    return w,b

In [63]:
# 파이토치 신경망 구현 

import torch.nn as nn

f = nn.Linear(17,1) 
f

# y_pred = torch.matmul(x,w)+b 보다 훨씬 단순해진 코드

Linear(in_features=17, out_features=1, bias=True)

# Loading data(Minibatch Gradient Descent에서)

## Dataset 클래스

- Dataset 클래스를 상속해 __사용자 정의 데이터셋 클래스를 만든다.__

In [64]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self): # 데이터셋 클래스의 테이블 읽기 / 이미지 파일명 읽기와 같은 초기화 작업이 필요할때, 이 메서드에 구현하기
        self.x_data = [[73,80,75],
                      [93,88,93],
                      [89,91,90],
                      [96,98,100],
                      [73,66,70]]
        self.y_data = [[152],[185],[180],[196],[142]]
        
    def __len__(self): # 이 데이터셋의 총 데이터수를 반환
        return len(self.x_data)
    
    def __getitem__(self, idx): #어떤 idx를 받았을때 그에 상응하는 입출력 데이터 반환 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
                              
        return x, y
    
dataset = CustomDataset()

## DataLoader 클래스

- 배치로 만드는 작업을 돕는 유틸리티 클래스

In [49]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size = 2, shuffle=True) #가져올 데이터셋, 배치사이즈, 셔플 

## Multivariate LR 예제

In [50]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력차원은 3이고 출력차원은 1

    def forward(self, x):
        return self.linear(x) # hypothesis 계산을 어떻게 할지 알려주는

In [51]:
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [52]:
n_epochs = 20

for epoch in range(n_epochs + 1):
    for batch_idx, samples in enumerate(dataloader): #인덱스와 가져온 데이터를 받음
        x_train, y_train = samples
        
        #hypothesis 계산
        prediction = model(x_train)
        
        #cost계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 hypothesis 업데이트
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {}/{} Batch {}{} Cost: {:6f}'.format(epoch, n_epochs, batch_idx+1, len(dataloader), cost.item()))

Epoch 0/20 Batch 13 Cost: 37811.089844
Epoch 0/20 Batch 23 Cost: 5945.278809
Epoch 0/20 Batch 33 Cost: 1988.551514
Epoch 1/20 Batch 13 Cost: 1036.760986
Epoch 1/20 Batch 23 Cost: 314.877808
Epoch 1/20 Batch 33 Cost: 124.283913
Epoch 2/20 Batch 13 Cost: 32.047115
Epoch 2/20 Batch 23 Cost: 13.070057
Epoch 2/20 Batch 33 Cost: 0.095560
Epoch 3/20 Batch 13 Cost: 5.376614
Epoch 3/20 Batch 23 Cost: 0.353518
Epoch 3/20 Batch 33 Cost: 1.954518
Epoch 4/20 Batch 13 Cost: 1.698066
Epoch 4/20 Batch 23 Cost: 2.195828
Epoch 4/20 Batch 33 Cost: 0.568628
Epoch 5/20 Batch 13 Cost: 1.189131
Epoch 5/20 Batch 23 Cost: 1.036901
Epoch 5/20 Batch 33 Cost: 2.770679
Epoch 6/20 Batch 13 Cost: 2.777880
Epoch 6/20 Batch 23 Cost: 1.978745
Epoch 6/20 Batch 33 Cost: 0.049007
Epoch 7/20 Batch 13 Cost: 1.866581
Epoch 7/20 Batch 23 Cost: 1.647989
Epoch 7/20 Batch 33 Cost: 0.740336
Epoch 8/20 Batch 13 Cost: 1.209825
Epoch 8/20 Batch 23 Cost: 0.416578
Epoch 8/20 Batch 33 Cost: 2.941945
Epoch 9/20 Batch 13 Cost: 0.485077
E